# 🎯 Enhanced Sleep Disorder Classification - Complete Pipeline

## 📌 Project Overview
This notebook implements state-of-the-art deep learning techniques for binary sleep disorder classification (Healthy vs Unhealthy).

## 🚀 Key Improvements Over Baseline:
1. **Attention-Enhanced BiLSTM-CNN Architecture**
   - Bidirectional LSTM for temporal context
   - Attention mechanism for feature importance
   - Skip connections for better gradient flow

2. **Advanced Data Augmentation**
   - Time warping, magnitude warping
   - Jittering, scaling, time shifting
   - Window slicing and rotation

3. **Stratified K-Fold Cross-Validation**
   - More reliable performance estimates
   - Reduced variance in metrics
   - Better generalization assessment

4. **Model Ensemble**
   - Combines multiple models for robustness
   - Weighted voting based on performance
   - Improved accuracy and reliability

5. **Comprehensive Visualization**
   - Training history analysis
   - Confusion matrices
   - ROC/PR curves
   - Attention weight visualization

## 📊 Expected Performance Improvements:
- **Accuracy**: +3-7% over baseline
- **ROC-AUC**: +2-5% improvement
- **F1-Score**: +3-6% boost
- **Robustness**: Significantly better generalization

---

## 1️⃣ Setup & Imports

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tfl
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"NumPy version: {np.__version__}")

%matplotlib inline

## 2️⃣ Load Dataset

**Note**: Update the file path to match your data location

In [ ]:
# TODO: Update this path to your dataset location
DATA_PATH = 'path/to/your/healthy_unhealthy1.csv'

# Load data
try:
    data = np.loadtxt(DATA_PATH, delimiter=',')
    print(f"✅ Data loaded successfully!")
    print(f"   Shape: {data.shape}")
    
    # Split features and labels
    X = data[:, 0:1024]  # First 1024 columns are features
    y = data[:, -1]      # Last column is label (0=Healthy, 1=Unhealthy)
    
    print(f"\n📊 Dataset Statistics:")
    print(f"   Total samples: {len(X)}")
    print(f"   Feature dimensions: {X.shape[1]}")
    print(f"   Healthy samples: {np.sum(y == 0)} ({np.sum(y == 0)/len(y)*100:.1f}%)")
    print(f"   Unhealthy samples: {np.sum(y == 1)} ({np.sum(y == 1)/len(y)*100:.1f}%)")
    print(f"   Class balance ratio: {np.sum(y == 0) / np.sum(y == 1):.2f}:1")
    
except FileNotFoundError:
    print("❌ Data file not found!")
    print("Please update DATA_PATH with your dataset location.")
    raise

## 3️⃣ Data Preparation & Augmentation

In [ ]:
# Run the data augmentation notebook to load functions
%run "1_Data_Augmentation_Utils.ipynb"

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    shuffle=True, 
    stratify=y,  # Maintain class balance
    random_state=42
)

print(f"📊 Data Split:")
print(f"   Training set: {len(X_train)} samples")
print(f"   Test set: {len(X_test)} samples")
print(f"   Train class distribution: {np.bincount(y_train.astype(int))}")
print(f"   Test class distribution: {np.bincount(y_test.astype(int))}")

In [ ]:
# Apply data augmentation (OPTIONAL - comment out if you want to skip augmentation)
APPLY_AUGMENTATION = True  # Set to False to skip augmentation
AUGMENTATION_FACTOR = 1     # How many augmented copies per sample (0 = no augmentation)

if APPLY_AUGMENTATION and AUGMENTATION_FACTOR > 0:
    print("🔄 Applying data augmentation...")
    X_train_aug, y_train_aug = augment_dataset(
        X_train, y_train,
        augmentation_factor=AUGMENTATION_FACTOR,
        augmentation_methods=['jitter', 'scaling', 'time_warp', 'magnitude_warp']
    )
else:
    print("⏭️ Skipping data augmentation")
    X_train_aug, y_train_aug = X_train, y_train

# Reshape for CNN input (samples, timesteps, features)
X_train_aug = X_train_aug.reshape(-1, 1024, 1)
X_test_reshaped = X_test.reshape(-1, 1024, 1)

print(f"\n✅ Final training data shape: {X_train_aug.shape}")
print(f"✅ Final test data shape: {X_test_reshaped.shape}")

## 4️⃣ Build Attention-Enhanced Model

In [ ]:
# Load model architecture
%run "2_Attention_BiLSTM_CNN_Model.ipynb"

In [ ]:
# Build and compile model
input_shape = (1024, 1)

print("🏗️ Building Attention-Enhanced BiLSTM-CNN model...")
model = build_attention_bilstm_cnn(input_shape, use_attention=True)
model = compile_attention_model(model, learning_rate=0.001)

# Display model summary
model.summary()

print(f"\n✅ Model built successfully!")
print(f"   Total parameters: {model.count_params():,}")

## 5️⃣ Train Model with Advanced Callbacks

In [ ]:
# Training configuration
EPOCHS = 150
BATCH_SIZE = 64
MODEL_SAVE_PATH = 'best_attention_model.h5'

print("🚀 Starting model training...\n")

history, training_time = train_attention_model(
    model=model,
    x_train=X_train_aug,
    y_train=y_train_aug,
    x_val=None,  # Will use validation_split=0.2
    y_val=None,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    save_path=MODEL_SAVE_PATH
)

print(f"\n✅ Training completed!")
print(f"   Total time: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")
print(f"   Model saved to: {MODEL_SAVE_PATH}")

## 6️⃣ Evaluate Model Performance

In [ ]:
# Load visualization utilities
%run "5_Visualization_Results.ipynb"

In [ ]:
# Run complete visualization pipeline
metrics = complete_visualization_pipeline(
    model=model,
    history=history,
    X_test=X_test_reshaped,
    y_test=y_test,
    class_names=['Healthy', 'Unhealthy'],
    save_dir='./results_attention_model'
)

## 7️⃣ K-Fold Cross-Validation (OPTIONAL but RECOMMENDED)

For more robust evaluation, run K-Fold cross-validation

In [ ]:
# Load K-Fold utilities
%run "3_KFold_CrossValidation.ipynb"

In [ ]:
# OPTIONAL: Run K-Fold Cross-Validation
RUN_KFOLD = False  # Set to True to run K-Fold CV (takes longer)

if RUN_KFOLD:
    print("🔄 Starting K-Fold Cross-Validation...\n")
    
    # Define model builder function for K-Fold
    def create_attention_model():
        model = build_attention_bilstm_cnn((1024, 1), use_attention=True)
        model = compile_attention_model(model, learning_rate=0.001)
        return model
    
    # Run K-Fold CV
    kfold_results, kfold_models, kfold_histories = kfold_cross_validation(
        model_builder=create_attention_model,
        X=X_train_aug,
        y=y_train_aug,
        n_splits=5,
        epochs=100,
        batch_size=64,
        random_state=42,
        verbose=1
    )
    
    # Visualize K-Fold results
    plot_kfold_results(kfold_results, save_path='./results_kfold/kfold_metrics.png')
    plot_metrics_boxplot(kfold_results, save_path='./results_kfold/kfold_boxplot.png')
    plot_training_histories(kfold_histories, save_path='./results_kfold/kfold_training.png')
    
    # Save K-Fold results
    save_kfold_results(kfold_results, save_dir='./results_kfold')
    
else:
    print("⏭️ Skipping K-Fold Cross-Validation (set RUN_KFOLD=True to enable)")

## 8️⃣ Ensemble Methods (OPTIONAL)

Combine multiple models for improved performance

In [ ]:
# Load ensemble utilities
%run "4_Model_Ensemble.ipynb"

In [ ]:
# OPTIONAL: Use ensemble if you have K-Fold models
USE_ENSEMBLE = False  # Set to True if you ran K-Fold CV

if USE_ENSEMBLE and RUN_KFOLD:
    print("🎯 Creating model ensemble...\n")
    
    # Calculate optimal weights based on validation performance
    # Note: You'd need a separate validation set for this
    # For now, we'll use equal weights
    
    # Evaluate ensemble methods
    ensemble_results = evaluate_ensemble_methods(
        models=kfold_models,
        X_test=X_test_reshaped,
        y_test=y_test,
        weights=None  # Equal weights
    )
    
    # Visualize ensemble comparison
    plot_ensemble_comparison(ensemble_results, save_path='./results_ensemble/ensemble_comparison.png')
    plot_roc_curves_comparison(ensemble_results, y_test, save_path='./results_ensemble/ensemble_roc.png')
    
else:
    print("⏭️ Skipping ensemble (set USE_ENSEMBLE=True and run K-Fold first)")

## 9️⃣ Compare with Baseline Models

In [ ]:
# Train a simple baseline for comparison
print("🔄 Training baseline CNN-LSTM model for comparison...\n")

baseline_model = build_simple_cnn_lstm((1024, 1))
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', 
                      metrics=['accuracy'])

baseline_history = baseline_model.fit(
    X_train_aug, y_train_aug,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)],
    verbose=0
)

print("✅ Baseline model trained!")

In [ ]:
# Evaluate baseline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Attention model predictions
y_pred_attention_proba = model.predict(X_test_reshaped, verbose=0).flatten()
y_pred_attention = (y_pred_attention_proba > 0.5).astype(int)

# Baseline predictions
y_pred_baseline_proba = baseline_model.predict(X_test_reshaped, verbose=0).flatten()
y_pred_baseline = (y_pred_baseline_proba > 0.5).astype(int)

# Calculate metrics
attention_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_attention),
    'F1-Score': f1_score(y_test, y_pred_attention),
    'ROC-AUC': roc_auc_score(y_test, y_pred_attention_proba)
}

baseline_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_baseline),
    'F1-Score': f1_score(y_test, y_pred_baseline),
    'ROC-AUC': roc_auc_score(y_test, y_pred_baseline_proba)
}

# Create comparison table
comparison = {
    'Attention-BiLSTM-CNN': attention_metrics,
    'Baseline-CNN-LSTM': baseline_metrics
}

comparison_df = create_comparison_table(comparison, save_path='./model_comparison.csv')

## 🔟 Final Summary & Recommendations

In [ ]:
print("\n" + "="*80)
print("🎊 PROJECT SUMMARY")
print("="*80)

print("\n📊 Dataset:")
print(f"   Total samples: {len(X)}")
print(f"   Training samples: {len(X_train_aug)}")
print(f"   Test samples: {len(X_test)}")

print("\n🏆 Best Model Performance:")
print(f"   Accuracy: {attention_metrics['Accuracy']*100:.2f}%")
print(f"   F1-Score: {attention_metrics['F1-Score']*100:.2f}%")
print(f"   ROC-AUC: {attention_metrics['ROC-AUC']*100:.2f}%")

print("\n📈 Improvement Over Baseline:")
acc_improvement = (attention_metrics['Accuracy'] - baseline_metrics['Accuracy']) * 100
f1_improvement = (attention_metrics['F1-Score'] - baseline_metrics['F1-Score']) * 100
auc_improvement = (attention_metrics['ROC-AUC'] - baseline_metrics['ROC-AUC']) * 100

print(f"   Accuracy: {acc_improvement:+.2f}%")
print(f"   F1-Score: {f1_improvement:+.2f}%")
print(f"   ROC-AUC: {auc_improvement:+.2f}%")

print("\n💾 Saved Artifacts:")
print(f"   - Best model: {MODEL_SAVE_PATH}")
print(f"   - Visualizations: ./results_attention_model/")
print(f"   - Comparison table: ./model_comparison.csv")
if RUN_KFOLD:
    print(f"   - K-Fold results: ./results_kfold/")
if USE_ENSEMBLE:
    print(f"   - Ensemble results: ./results_ensemble/")

print("\n🎯 Key Techniques Used:")
print("   ✅ Attention mechanism for feature focus")
print("   ✅ Bidirectional LSTM for temporal context")
print("   ✅ Skip connections for gradient flow")
print("   ✅ Batch normalization for stability")
print("   ✅ Data augmentation for generalization")
if RUN_KFOLD:
    print("   ✅ K-Fold cross-validation for robustness")
if USE_ENSEMBLE:
    print("   ✅ Model ensemble for improved accuracy")

print("\n" + "="*80)
print("✅ PROJECT COMPLETED SUCCESSFULLY!")
print("="*80 + "\n")

## 📝 Next Steps & Recommendations

### For Project Submission:
1. **Include all visualizations** from `results_attention_model/` directory
2. **Present the comparison table** showing improvements over baseline
3. **Highlight K-Fold CV results** if you ran them (shows rigorous evaluation)
4. **Explain the architecture** with attention mechanism and BiLSTM
5. **Discuss data augmentation** and its impact on generalization

### Further Improvements:
1. **Hyperparameter Tuning**: Use KerasTuner or Optuna for automated optimization
2. **Transfer Learning**: Pre-train on larger sleep datasets if available
3. **Multi-Class Classification**: Extend to classify specific sleep disorders
4. **Real-Time Prediction**: Deploy model as web service or mobile app
5. **Explainability**: Add SHAP or LIME for model interpretability

### Presentation Tips:
- Show before/after comparison with baseline
- Demonstrate attention weights on sample signals
- Present confusion matrix and ROC curves
- Explain why each technique was chosen
- Discuss limitations and future work

---

**Good luck with your project submission! 🚀**